# Vector, Time Series and Graph Data Representations of Multivariate Time Series Data for Photospheric Magnetic Field Parameter-based Solar Flare Classification

## *by Onur Vural*

### This module will allow a step by step guide to run all experiments and document results in a single notebook file
--Version h01

-Prerequisites: To use the partitions, you need to download the SWAN-SF partitions from: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/EBCFKM
After downloading, the partitions have to be placed under folder DATA with their original names

-As the experiments are being made, the models will be saved under folder MODELS





# Processing of data  

In [ ]:
# Packages required
import os
import pandas as pd
from natsort import natsorted
from pathlib import Path

# M: Specify the partition to use
# Partitions have to be adjacent
partitions = ['partition1', 'partition2', 'partition3', 'partition4', 'partition5' ]
training_partition = partitions[3]
testing_partition = partitions[4]
print("Training with ", training_partition, " and testing with ", testing_partition)

cur_dir = Path.cwd()
parent_folder = cur_dir.parent / 'DATA' 

# Defining the paths
folder_path_flare_train = parent_folder / training_partition / 'FL'
folder_path_nonflare_train = parent_folder / training_partition / 'NF'
folder_path_flare_test = parent_folder / testing_partition / 'FL'
folder_path_nonflare_test = parent_folder / testing_partition / 'NF'

In [ ]:
def process_data(folder_path, is_nonflare_train=False):
    data_list = []
    
    if is_nonflare_train:
        csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file.startswith('FQ')]
    else:
        csv_files = natsorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])
 
    # Loop through the sorted CSV files and load them into dataframes
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        # Use the file name (without extension) as the key in the dictionary
        df = pd.read_csv(file_path, sep='\t')
        # Extract magnetic field parameter values
        first_24_features = df.iloc[:, 1:25]
        data_list.append(first_24_features)
    
    # Detect nan entries
    nan_count = 0
    indexes_with_nan = []
    for index, df in enumerate(data_list):
        if df.isna().any().any():
            if index not in indexes_with_nan:
                nan_count += 1
                indexes_with_nan.append(index)
    percentageNaN = (nan_count / len(data_list)) * 100
    
    # Discard nan entries
    data_list = [df for index, df in enumerate(data_list) if index not in indexes_with_nan]
    
    return data_list, percentageNaN

In [ ]:
# Call the functions (may take long!)
flare_data_list_train, percentageNaN = process_data(folder_path_flare_train)
print("Number of DataFrames with NaN values among train flare examples: ", percentageNaN)
nonflare_data_list_train, percentageNaN  = process_data(folder_path_nonflare_train, True)
print("Number of DataFrames with NaN values among train nonflare examples: ", percentageNaN)
flare_data_list_test, percentageNaN  = process_data(folder_path_flare_test)
print("Number of DataFrames with NaN values among test flare examples: ", percentageNaN)
nonflare_data_list_test, percentageNaN  = process_data(folder_path_nonflare_test)
print("Number of DataFrames with NaN values among test nonflare examples: ", percentageNaN)

# Method 2: TS Last Instance - Traditional Classifiers

In [ ]:
import numpy as np
import pandas as pd

def extractLastInstance(flare_data_list, non_flare_data_list):
    flare_labels = np.ones(len(flare_data_list))
    non_flare_labels = np.zeros(len(non_flare_data_list))
    # Combine examples
    ts_data_list = flare_data_list + non_flare_data_list
    ts_labels = np.concatenate([flare_labels, non_flare_labels])
    # shuffle the data
    permutation = np.random.permutation(len(ts_data_list))
    ts_data_list = [ts_data_list[i] for i in permutation]
    ts_labels = ts_labels[permutation]

    # Convert df objects to np array format
    ts_data_list = np.array([ts.to_numpy() for ts in ts_data_list])
    
    # Extract the last column for every TS 
    ts_data_list_LAST = ts_data_list[:, -1, :]   
    return ts_data_list, ts_data_list_LAST, ts_labels 

In [ ]:
ts_data_list_train, ts_data_list_LAST_train, ts_labels_train = extractLastInstance(flare_data_list_train, nonflare_data_list_train)
ts_data_list_test, ts_data_list_LAST_test, ts_labels_test = extractLastInstance(flare_data_list_test, nonflare_data_list_test)

In [ ]:
import pickle
import os
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

classifiers = ['SVC', 'KNC', 'DT', 'LR', 'MLP']
def train_test_classifier_method2(ts_data_list_train_LAST, ts_labels_train, ts_data_list_test_LAST, ts_labels_test, classifier_name):
    # Create and train the classifier
    if classifier_name == 'SVC':
        classifier = SVC(class_weight='balanced')
    elif classifier_name == 'KNC':
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif classifier_name == 'DT':
        classifier = DecisionTreeClassifier(random_state=42)
    elif classifier_name == 'LR':
        classifier = LogisticRegression(max_iter=100000)
    elif classifier_name == 'MLP':
        classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000) 
    else :
        classifier = SVC(class_weight='balanced')
    # train selected classifier        
    classifier.fit(ts_data_list_train_LAST, ts_labels_train)
    # test selected classifier on unseen data
    predictions = classifier.predict(ts_data_list_test_LAST)

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(ts_labels_test, predictions)

    # Calculate True Positives, True Negatives, False Positives, and False Negatives
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Calculate True Skill Statistics (TSS)
    tss = (tp / (tp + fn)) - (fp / (fp + tn))

    # Calculate Heidke Skill Score 1 (HSS1)
    hss1 = (tp / (tp + fn) )*(2 - (tp + fp)/tp )

    # Calculate Heidke Skill Score 2 (HSS2)
    hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

    # Calculate F1 Score
    f1 = f1_score(ts_labels_test, predictions)

    # Calculate Gilbert Skill Score
    gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn))

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(ts_labels_test, predictions)
    
    # save the model
    cur_dir = Path.cwd()
    model_name = training_partition + '_' + testing_partition + '_' + classifier_name + '_model_tsLAST.pkl'
    model_path = cur_dir.parent / 'MODELS' / model_name
    
    with open(model_path, 'wb') as model_file:
        pickle.dump(classifier, model_file)

    print("Saved ", classifier_name," to", model_path)
    # Print the results
    print(classifier_name, " Model Evaluation Metrics:")
    print("Accuracy:", accuracy)
    print("True Skill Statistics (TSS):", tss)
    print("Heidke Skill Score 1 (HSS1):", hss1)
    print("Heidke Skill Score 2 (HSS2):", hss2)
    print("F1 Score:", f1)
    print("Gilbert Skill Score:", gilbert)
    print("ROC AUC Score:", roc_auc)
    print("********************************************")

In [ ]:
# Train all classifiers
for classifier_name in classifiers:
    train_test_classifier_method2(ts_data_list_LAST_train, ts_labels_train, ts_data_list_LAST_test, ts_labels_test, classifier_name)    

# Method 3: TS Classification

In [ ]:
import pickle
import os
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.kernel_based import RocketClassifier

classifiers_ts = ['ST','TSFC','ROCKET']
def train_test_classifier_method3(ts_data_list_train, ts_labels_train, ts_data_list_test, ts_labels_test, classifier_name):
    
    # Create and train the classifier
    if classifier_name == 'ST':
        classifier = ShapeletTransformClassifier(
                        estimator=RotationForest(n_estimators=3),
                        n_shapelet_samples=100,
                        max_shapelets=10,
                        batch_size=20)
        classifier.fit(ts_data_list_train, ts_labels_train)
        predictions = classifier.predict(ts_data_list_test)
    elif classifier_name == 'TSFC':
        ts_data_list_train_concatenated = ts_data_list_train.reshape((len(ts_data_list_train), -1))
        classifier = TimeSeriesForestClassifier(n_estimators=100, random_state=42)
        classifier.fit(ts_data_list_train_concatenated, ts_labels_train)
        ts_data_list_test_concatenated = ts_data_list_test.reshape((len(ts_data_list_test), -1))    
        predictions = classifier.predict(ts_data_list_test_concatenated)
    elif classifier_name == 'ROCKET':
        classifier = RocketClassifier(num_kernels=500)
        classifier.fit(ts_data_list_train, ts_labels_train)
        predictions = classifier.predict(ts_data_list_test)
    else:
        classifier = RocketClassifier(num_kernels=500)
        classifier.fit(ts_data_list_train, ts_labels_train)
        predictions = classifier.predict(ts_data_list_test)

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(ts_labels_test, predictions)

    # Calculate True Positives, True Negatives, False Positives, and False Negatives
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Calculate True Skill Statistics (TSS)
    tss = (tp / (tp + fn)) - (fp / (fp + tn))

    # Calculate Heidke Skill Score 1 (HSS1)
    hss1 = (tp / (tp + fn) )*(2 - (tp + fp)/tp )

    # Calculate Heidke Skill Score 2 (HSS2)
    hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

    # Calculate F1 Score
    f1 = f1_score(ts_labels_test, predictions)

    # Calculate Gilbert Skill Score
    gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn))

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(ts_labels_test, predictions)
    
    # save the model
    cur_dir = Path.cwd()
    model_name = training_partition + '_' + testing_partition + '_' + classifier_name + '_model_ts.pkl'
    model_path = cur_dir.parent / 'MODELS' / model_name
    
    with open(model_path, 'wb') as model_file:
        pickle.dump(classifier, model_file)

    print("Saved ", classifier_name," to", model_path)
    # Print the results
    print(classifier_name, " Model Evaluation Metrics:")
    print("Accuracy:", accuracy)
    print("True Skill Statistics (TSS):", tss)
    print("Heidke Skill Score 1 (HSS1):", hss1)
    print("Heidke Skill Score 2 (HSS2):", hss2)
    print("F1 Score:", f1)
    print("Gilbert Skill Score:", gilbert)
    print("ROC AUC Score:", roc_auc)
    print("********************************************")

In [ ]:
# Train and save all classifiers
for classifier_name in classifiers_ts:
    train_test_classifier_method3(ts_data_list_train, ts_labels_train, ts_data_list_test, ts_labels_test, classifier_name)

# LSTM

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data

# Assuming you have ts_data_list_train, ts_labels_train, ts_data_list_test, ts_labels_test

class TimeSeriesDataset(Dataset):
    def __init__(self, data_list, labels):
        self.data_list = data_list
        self.labels = labels

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        data = torch.Tensor(self.data_list[index])  # Assuming your data is already in tensor format
        label = torch.Tensor([self.labels[index]])
        return {'x': data, 'y': label}

# Create datasets and loaders
train_dataset = TimeSeriesDataset(ts_data_list_train, ts_labels_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TimeSeriesDataset(ts_data_list_test, ts_labels_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Assuming you have train_loader and test_loader already defined

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1, :, :])
        return torch.sigmoid(out)

# Initialize LSTM model
input_size = 24  # Assuming each time step has 24 features
hidden_size = 64
num_layers = 2
output_size = 1  # Binary classification

lstm_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Initialize LSTM model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    lstm_model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        data, labels = batch['x'], batch['y']
        optimizer.zero_grad()
        output = lstm_model(data)
        loss = criterion(output, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

        # Update metrics
        total_loss += loss.item()
        total_samples += len(labels) 
        predicted_labels = (output >= 0.5).float()
        correct_predictions += (predicted_labels == labels.view(-1, 1)).sum().item()

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    train_losses.append(average_loss)
    train_accuracies.append(accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
cur_dir = Path.cwd()
model_name = training_partition + '_' + testing_partition + '_LSTM_model_ts.pkl'
model_path = cur_dir.parent / 'MODELS' / model_name
torch.save(lstm_model.state_dict(), model_path)
print(f"Saved LSTM model to {model_path}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch vs Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epoch vs Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on the test set
lstm_model.eval()
true_labels, predicted_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        data, labels = batch['x'], batch['y']
        output = (lstm_model(data) >= 0.5).float()
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(output.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate True Skill Statistics (TSS)
tss = (tp / (tp + fn)) - (fp / (fp + tn))

# Calculate Heidke Skill Score 1 (HSS1)
hss1 = (tp / (tp + fn)) * (2 - (tp + fp) / tp)

# Calculate Heidke Skill Score 2 (HSS2)
hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

# Calculate F1 Score
f1 = f1_score(true_labels, predicted_labels)

# Calculate Gilbert Skill Score
gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (
        tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)
)

# Calculate ROC AUC score
roc_auc = roc_auc_score(true_labels, predicted_labels)

# Print the results
print("LSTM Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("True Skill Statistics (TSS):", tss)
print("Heidke Skill Score 1 (HSS1):", hss1)
print("Heidke Skill Score 2 (HSS2):", hss2)
print("F1 Score:", f1)
print("Gilbert Skill Score:", gilbert)
print("ROC AUC Score:", roc_auc)

# RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score
  
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        _, hidden = self.rnn(x)
        out = self.fc(hidden[-1, :, :])  # Take the hidden state of the last time step
        out = self.sigmoid(out)
        return out

# Hyperparameters
input_size = 24  # Number of features
hidden_size = 64  # Number of hidden units
output_size = 1  # Output size (1 for binary classification)

# Create an instance of the RNN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rnn_model = RNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)   
criterion = nn.BCELoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    rnn_model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        data, labels = batch['x'], batch['y']
        optimizer.zero_grad()
        output = rnn_model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # Update metrics
        total_loss += loss.item()
        total_samples += labels.size(0)
        predicted_labels = (output >= 0.5).float()
        correct_predictions += (predicted_labels == labels).sum().item()

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    train_losses.append(average_loss)
    train_accuracies.append(accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
cur_dir = Path.cwd()
model_name = training_partition + '_' + testing_partition + '_RNN_model_ts.pkl'
model_path = cur_dir.parent / 'MODELS' / model_name
torch.save(rnn_model.state_dict(), model_path)
print(f"Saved RNN model to {model_path}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch vs Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epoch vs Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on the test set
rnn_model.eval()
true_labels, predicted_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        data, labels = batch['x'], batch['y']
        output = (rnn_model(data) >= 0.5).float()
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(output.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate True Skill Statistics (TSS)
tss = (tp / (tp + fn)) - (fp / (fp + tn))

# Calculate Heidke Skill Score 1 (HSS1)
hss1 = (tp / (tp + fn)) * (2 - (tp + fp) / tp)

# Calculate Heidke Skill Score 2 (HSS2)
hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

# Calculate F1 Score
f1 = f1_score(true_labels, predicted_labels)

# Calculate Gilbert Skill Score
gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (
        tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)
)

# Calculate ROC AUC score
roc_auc = roc_auc_score(true_labels, predicted_labels)

# Print the results
print("RNN Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("True Skill Statistics (TSS):", tss)
print("Heidke Skill Score 1 (HSS1):", hss1)
print("Heidke Skill Score 2 (HSS2):", hss2)
print("F1 Score:", f1)
print("Gilbert Skill Score:", gilbert)
print("ROC AUC Score:", roc_auc)

# METHOD 1: Graph-based Classification 

Creation of Pearson correlation matrices

In [ ]:
import numpy as np
import os

cur_dir = Path.cwd()
correlation_folder = cur_dir.parent / 'DATA' / 'pearson_correlation'

def calculate_correlation_matrix(data_list, is_flare, partition_name, train_or_test):
    if is_flare:
        name = 'flare'
    else:
        name = 'nonflare'
        
    f_name = partition_name + '_' + train_or_test
    new_folder_path = os.path.join(correlation_folder, f_name)
    # Check whether the folder exists first, if not, create it
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        
    # Iterate through the list of DataFrames in data_list
    for index, df in enumerate(data_list):
        # Compute the correlation matrix for the DataFrame object
        correlation_matrix = df.corr()
        
        # Remove self-loops
        np.fill_diagonal(correlation_matrix.values, 0)
        
        # Threshold by zero
        correlation_matrix[correlation_matrix < 0] = 0
    
        # Define the file path for saving the correlation matrix
        file_path = os.path.join(new_folder_path, f"{name}_correlation_matrix_{index}.csv")
        
        # Save the correlation matrix to the specified file path
        correlation_matrix.to_csv(file_path, index=True, header=True)

In [ ]:
# Create correlation matrices for train and test partitions and save them into their respective folders
# train
calculate_correlation_matrix(flare_data_list_train, is_flare=True, 
                             partition_name=training_partition, train_or_test='train')
calculate_correlation_matrix(nonflare_data_list_train, is_flare=False, 
                             partition_name=training_partition, train_or_test='train')

In [ ]:
# test
calculate_correlation_matrix(flare_data_list_test, is_flare=True, 
                             partition_name=testing_partition, train_or_test='test')
calculate_correlation_matrix(nonflare_data_list_test, is_flare=False, 
                             partition_name=testing_partition, train_or_test='test')

Creation of Euclidian distance matrices

In [ ]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

cur_dir = Path.cwd()
euclidian_folder = cur_dir.parent / 'DATA' / 'euclidian_distance'
# For z-Normalization
scaler = StandardScaler() 

def calculate_euclidian_matrix(data_list, is_flare, partition_name, train_or_test):
    if is_flare:
        name = 'flare'
    else:
        name = 'nonflare'
    
    f_name = partition_name + '_' + train_or_test
    new_folder_path = os.path.join(euclidian_folder, f_name)
    # Check whether the folder exists first, if not, create it
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        
    # Iterate through the list of DataFrames in data_list
    for index, df in enumerate(data_list):
        row_names = df.columns
        col_names = df.columns
        ft_num = df.shape[1]
        euclidian_distances = np.zeros((ft_num, ft_num))
    
        # NORMALIZE THE DATA COLUMNWISE BEFORE DISTANCE CALCULATION
        normalized_df = scaler.fit_transform(df)
        # Create a new DataFrame with the normalized values
        normalized_df = pd.DataFrame(normalized_df, columns=df.columns, index=df.index)

        for i in range(ft_num):
            for j in range(i, ft_num):
                # Feature i and j and j: euclidean distance between i and j
                euclidian_distance = np.linalg.norm(normalized_df.iloc[:, i] - normalized_df.iloc[:, j])
                euclidian_distances[i, j] = euclidian_distance
                euclidian_distances[j, i] = euclidian_distance

        # Save the Euclidean distances as a CSV file
        euclidian_distances_df = pd.DataFrame(euclidian_distances, index=row_names, columns=col_names)
        dist_file_path = os.path.join(new_folder_path, f"{name}_euclidian_matrix_{index}.csv")
        euclidian_distances_df.to_csv(dist_file_path)     

In [ ]:
# Create euclidian matrices for train and test partitions and save them into their respective folders
# train
calculate_euclidian_matrix(flare_data_list_train, is_flare=True, 
                           partition_name=training_partition, train_or_test='train')
calculate_euclidian_matrix(nonflare_data_list_train, is_flare=False, 
                           partition_name=training_partition, train_or_test='train')

In [ ]:
# test
calculate_euclidian_matrix(flare_data_list_test, is_flare=True, 
                           partition_name=testing_partition, train_or_test='test')
calculate_euclidian_matrix(nonflare_data_list_test, is_flare=False, 
                           partition_name=testing_partition, train_or_test='test')

Creation of graphs

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import pickle
from pathlib import Path


cur_dir = Path.cwd()
matrix_folder = cur_dir.parent / 'DATA' # For input matrix, the type will be specified
graph_folder = cur_dir.parent / 'DATA' / 'graphs' # For output graphs
graph_categories = ['pearson_correlation','euclidian_distance']

def create_graphs(is_flare, partition_name, graph_category, train_or_test):
    if is_flare:
        name = 'flare'
    else:
        name = 'nonflare'    
        
    f_name = partition_name + '_' + train_or_test
    input_folder_path = matrix_folder / graph_category / f_name
    new_folder_path = graph_folder / graph_category / f_name 
    # Check whether the folder exists first, if not, create it
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    
    # Iterate through the matrices, convert them into graph objects
    for file in os.listdir(input_folder_path):
        if file.endswith('.csv'):
            # Read the coMatrix
            file_path = os.path.join(input_folder_path, file)
            instance = pd.read_csv(file_path, index_col=0)
        
            if graph_category == 'pearson_correlation':
                # Create graph
                myG = nx.Graph(instance.values)
            elif graph_category == 'euclidian_distance':
                THRESHOLD_VALUE = 10
                myG = nx.Graph()
                # Add nodes for each feature in the DataFrame with their names
                for feature in instance.columns:
                    myG.add_node(feature)
                # Apply threshold principle when creating the edges
                n = len(instance.columns)
                for i in range(n):
                    for j in range(i + 1, n):
                        if np.abs(instance.iat[i, j]) < THRESHOLD_VALUE:
                            # Add an edge between the corresponding nodes and assign their names as attributes
                            node1 = instance.columns[i]
                            node2 = instance.columns[j]
                            myG.add_edge(node1, node2)
            # Save graph
            outputG_path = os.path.join(new_folder_path, f"{file}.graph")
            with open(outputG_path, 'wb') as f:
                pickle.dump(myG, f)

In [ ]:
# Call def to create desired graphs and save them
create_graphs(is_flare=True, partition_name=training_partition, 
              graph_category='pearson_correlation', train_or_test='train')
create_graphs(is_flare=False, partition_name=training_partition, 
              graph_category='pearson_correlation', train_or_test='train')

create_graphs(is_flare=True, partition_name=testing_partition, 
              graph_category='pearson_correlation', train_or_test='test')
create_graphs(is_flare=False, partition_name=testing_partition, 
              graph_category='pearson_correlation', train_or_test='test')

create_graphs(is_flare=True, partition_name=training_partition, 
              graph_category='euclidian_distance', train_or_test='train')
create_graphs(is_flare=False, partition_name=training_partition, 
              graph_category='euclidian_distance', train_or_test='train')

create_graphs(is_flare=True, partition_name=testing_partition, 
              graph_category='euclidian_distance', train_or_test='test')
create_graphs(is_flare=False, partition_name=testing_partition, 
              graph_category='euclidian_distance', train_or_test='test')

Processing of the graphs by extracting the degree information as a vector along with label information

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import pickle
from pathlib import Path
def process_graphs_m11(partition_name, graph_category, train_or_test):
    f_name = partition_name + '_' + train_or_test
    cur_dir = Path.cwd()
    graph_folder_input = cur_dir.parent / 'DATA' / 'graphs' # For specified graph path
    graph_folder_input = graph_folder_input / graph_category / f_name
    graph_vectors = []
    labels = []

    # Iterate through all files in the folder
    for filename in os.listdir(graph_folder_input):
        graph_path = os.path.join(graph_folder_input, filename)
        try:
            with open(graph_path, 'rb') as f:
                G = pickle.load(f)
            
            if graph_category == 'pearson_correlation': 
                # Compute node degrees
                degrees = [deg for node, deg in G.degree() if node != len(G.nodes) - 1]
            elif graph_category == 'euclidian_distance':
                # Compute node degrees
                degrees = [deg for node, deg in G.degree()]
           
            # Determine label based on filename
            if filename.startswith('flare'):
                label = 1
            elif filename.startswith('nonflare'):
                label = 0
        
            # Append degrees and label to lists
            graph_vectors.append(degrees)
            labels.append(label)
    
        except Exception as e:
            print(f"Error processing file {filename}: {str(e)}")

    # Convert lists to NumPy arrays
    graph_matrix = np.array(graph_vectors)
    label_vector = np.array(labels)
    print("Graph Matrix Shape:", graph_matrix.shape)
    print("Label Vector Shape:", label_vector.shape)
    return graph_matrix, label_vector

In [ ]:
graph_matrix_cor_train, label_vector_cor_train=process_graphs_m11(partition_name=training_partition, 
                                                                  graph_category='pearson_correlation', train_or_test='train')
graph_matrix_cor_test, label_vector_cor_test=process_graphs_m11(partition_name=testing_partition, 
                                                                graph_category='pearson_correlation', train_or_test='test')

graph_matrix_euc_train, label_vector_euc_train = process_graphs_m11(partition_name=training_partition, 
                                                                    graph_category='euclidian_distance', train_or_test='train')
graph_matrix_euc_test, label_vector_euc_test = process_graphs_m11(partition_name=testing_partition, 
                                                                  graph_category='euclidian_distance', train_or_test='test')

# Method 1.1: Degree Vector as it is

In [ ]:
import pickle
import os
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

classifiers = ['SVC', 'KNC', 'DT', 'LR', 'MLP']
def train_test_classifier_method11(graph_category, graph_matrix_train, label_vector_train, graph_matrix_test, label_vector_test, classifier_name):
    # Create and train the classifier
    if classifier_name == 'SVC':
        classifier = SVC(class_weight='balanced')
    elif classifier_name == 'KNC':
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif classifier_name == 'DT':
        classifier = DecisionTreeClassifier(random_state=42)
    elif classifier_name == 'LR':
        classifier = LogisticRegression(max_iter=10000)
    elif classifier_name == 'MLP':
        classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000) 
    else :
        classifier = SVC(class_weight='balanced')
    # train selected classifier        
    classifier.fit(graph_matrix_train, label_vector_train)
    # test selected classifier on unseen data
    predictions = classifier.predict(graph_matrix_test)

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(label_vector_test, predictions)

    # Calculate True Positives, True Negatives, False Positives, and False Negatives
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Calculate True Skill Statistics (TSS)
    tss = (tp / (tp + fn)) - (fp / (fp + tn))

    # Calculate Heidke Skill Score 1 (HSS1)
    hss1 = (tp / (tp + fn) )*(2 - (tp + fp)/tp )

    # Calculate Heidke Skill Score 2 (HSS2)
    hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

    # Calculate F1 Score
    f1 = f1_score(label_vector_test, predictions)

    # Calculate Gilbert Skill Score
    gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn))

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(label_vector_test, predictions)
    
    # save the model
    if graph_category == 'pearson_correlation':
        codeName = 'cor'
    elif graph_category == 'euclidian_distance':
        codeName = 'euc'
        
    cur_dir = Path.cwd()
    model_name = training_partition + '_' + testing_partition + '_' + classifier_name + '_model_' + codeName + '.pkl'
    model_path = cur_dir.parent / 'MODELS' / model_name
    
    with open(model_path, 'wb') as model_file:
        pickle.dump(classifier, model_file)

    print("Saved ", classifier_name," to", )
    # Print the results
    print(classifier_name, " Model Evaluation Metrics:")
    print("Accuracy:", accuracy)
    print("True Skill Statistics (TSS):", tss)
    print("Heidke Skill Score 1 (HSS1):", hss1)
    print("Heidke Skill Score 2 (HSS2):", hss2)
    print("F1 Score:", f1)
    print("Gilbert Skill Score:", gilbert)
    print("ROC AUC Score:", roc_auc)
    print("********************************************")

In [ ]:
# Train all classifiers (COR)
for classifier_name in classifiers:
    train_test_classifier_method11('pearson_correlation', graph_matrix_cor_train, label_vector_cor_train, 
                                   graph_matrix_cor_test, label_vector_cor_test, classifier_name)    

In [ ]:
# Train all classifiers (EUC)
for classifier_name in classifiers:
    train_test_classifier_method11('euclidian_distance', graph_matrix_euc_train, label_vector_euc_train, 
                                   graph_matrix_euc_test, label_vector_euc_test, classifier_name)   

# Method 1.2: Degree Vector with Node Embedding 

In [ ]:
import os
import pickle
import networkx as nx
import numpy as np
import random 

def process_graphs_m12(partition_name, graph_category, train_or_test):
    f_name = partition_name + '_' + train_or_test
    cur_dir = Path.cwd()
    graph_folder_input = cur_dir.parent / 'DATA' / 'graphs' # For specified graph path
    graph_folder_input = graph_folder_input / graph_category / f_name
    flare_graphs = []
    flare_labels = []
    nonflare_graphs = []
    nonflare_labels = []

    # Iterate through all files in the folder
    for filename in os.listdir(graph_folder_input):
        graph_path = os.path.join(graph_folder_input, filename)
        try:
            with open(graph_path, 'rb') as f:
                myG = pickle.load(f)
           
            # Determine label based on filename
            if filename.startswith('flare'):
                label = 1
                # only add to list if it is connected graph
                if nx.is_connected(myG): 
                    flare_graphs.append((myG, label))
                else:
                    continue
            elif filename.startswith('nonflare'):
                label = 0
                # only add to list if it is connected graph
                if nx.is_connected(myG): 
                    nonflare_graphs.append((myG, label))
                else:
                    continue
        except Exception as e:
            print(f"Error processing file {filename}: {str(e)}")

    graphs = flare_graphs + nonflare_graphs
    # Shuffle the balanced dataset
    random.shuffle(graphs)
    graph_list = [graph for graph, label in graphs]
    label_vector = [label for graph, label in graphs]
    label_vector = np.array(label_vector)    
    return graph_list, label_vector

In [ ]:
graph_list_cor_train, label_vector_cor_train = process_graphs_m12(partition_name=training_partition, 
                                                                  graph_category='pearson_correlation', train_or_test='train')
graph_list_cor_test, label_vector_cor_test = process_graphs_m12(partition_name=testing_partition, 
                                                                  graph_category='pearson_correlation', train_or_test='test')



In [ ]:
graph_list_euc_train, label_vector_euc_train = process_graphs_m12(partition_name=training_partition, 
                                                                  graph_category='euclidian_distance', train_or_test='train')
graph_list_euc_test, label_vector_euc_test = process_graphs_m12(partition_name=testing_partition, 
                                                                graph_category='euclidian_distance', train_or_test='test')

# Laplacian Embedding

In [ ]:
import pandas
from sklearn.preprocessing import StandardScaler
from scipy.sparse.linalg import eigsh

# Set dimension as 14 (75% representation power)
def laplacian_embedding(d, graph_list, graph_category):
    # For Z-normalization
    scaler = StandardScaler()
    
    graph_laplacian_node_embeddings_list = []
    
    # Iterate through each graph in graph_list
    for graph in graph_list:

        # Calculate the Laplacian
        L = nx.laplacian_matrix(graph).toarray()
        L = L.astype('float64')
        # Perform eigendecomposition of L
        eigenvalues, eigenvectors = eigsh(L, k=d, which='LM')
    

        # Sort eigenvalues and select the top d eigenvectors
        sorted_indices = np.argsort(eigenvalues)
        top_indices = sorted_indices[:d]
        embedding_matrix = eigenvectors[:, top_indices]
    
        normalized_embedding_matrix = scaler.fit_transform(embedding_matrix)
        graph_laplacian_node_embeddings_list.append(normalized_embedding_matrix)
        
    # flatten as 1-D feature respresentation
    flat_embeddings = [emb.flatten() for emb in graph_laplacian_node_embeddings_list]
    X_train = np.array(flat_embeddings)
    return X_train   

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Warning-causing lines of code here
    X_train_cor_lap = laplacian_embedding(14, graph_list_cor_train, graph_category='pearson_correlation')
    X_test_cor_lap = laplacian_embedding(14, graph_list_cor_test, graph_category='pearson_correlation')

    X_train_euc_lap = laplacian_embedding(14, graph_list_euc_train, graph_category='euclidian_distance')
    X_test_euc_lap = laplacian_embedding(14, graph_list_euc_test, graph_category='euclidian_distance')

# Node2Vec Embedding

In [ ]:
from node2vec import Node2Vec
import networkx as nx

class Node2Vec(Node2Vec):
  """
  Parameters
  ----------
  p : float
      p parameter of node2vec
  q : float
      q parameter of node2vec
  d : int
      dimensionality of the embedding vectors
  """
  def __init__(self, graph, p=1, q=1, d=32):
    super().__init__(
                     graph = graph,
                     walk_length=10,
                     p=p,
                     q=q,
                     dimensions =d
                  )

In [ ]:
from sklearn.preprocessing import StandardScaler

# Set dimension as 14 (75% representation power)
def node2vec_embedding(graph_list):
    # For Z-normalization
    scaler = StandardScaler()

    graph_node2vec_embeddings_list = []
    i = 0

    # Iterate through list of graphs
    for graph in graph_list:
        print("Example", i)
        # Apply Node2Vec to get embedding for each graph
        n2v_model = Node2Vec(graph, 1, 1, 10)
        model = n2v_model.fit(window=10, min_count=1, batch_words=4)
        # Node2Vec representation
        graph_node_embeddings = model.wv.vectors
            
        # Append the embeddings to the batch and their corresponding label
        graph_node2vec_embeddings_list.append(graph_node_embeddings)   
        i = i + 1

    # Proceed with flattening and creating feature matrix (X_train) 
    flat_embeddings = [emb.flatten() for emb in graph_node2vec_embeddings_list]
    X_train = np.array(flat_embeddings)
    return X_train

In [ ]:
X_train_cor_N2V = node2vec_embedding(graph_list_cor_train)

In [ ]:
X_test_cor_N2V = node2vec_embedding(graph_list_cor_test)

In [ ]:
X_train_euc_N2V = node2vec_embedding(graph_list_euc_train)

In [ ]:
X_test_euc_N2V = node2vec_embedding(graph_list_euc_test)

In [ ]:
import pickle
import os
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

classifiers = ['SVC', 'KNC', 'DT', 'LR', 'MLP']
def train_test_classifier_method12(graph_category, embedding_category, X_train, label_vector_train, X_test, 
                                   label_vector_test, classifier_name):
    # Create and train the classifier
    if classifier_name == 'SVC':
        classifier = SVC(class_weight='balanced')
    elif classifier_name == 'KNC':
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif classifier_name == 'DT':
        classifier = DecisionTreeClassifier(random_state=42)
    elif classifier_name == 'LR':
        classifier = LogisticRegression(max_iter=10000)
    elif classifier_name == 'MLP':
        classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000) 
    else:
        classifier = SVC(class_weight='balanced')
    # train selected classifier        
    classifier.fit(X_train, label_vector_train)
    # test selected classifier on unseen data
    predictions = classifier.predict(X_test)

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(label_vector_test, predictions)

    # Calculate True Positives, True Negatives, False Positives, and False Negatives
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Calculate True Skill Statistics (TSS)
    tss = (tp / (tp + fn)) - (fp / (fp + tn))

    # Calculate Heidke Skill Score 1 (HSS1)
    hss1 = (tp / (tp + fn) )*(2 - (tp + fp)/tp )

    # Calculate Heidke Skill Score 2 (HSS2)
    hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

    # Calculate F1 Score
    f1 = f1_score(label_vector_test, predictions)

    # Calculate Gilbert Skill Score
    gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn))

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(label_vector_test, predictions)
    
    # save the model
    if graph_category == 'pearson_correlation':
        graph_name = 'cor'
    elif graph_category == 'euclidian_distance':
        graph_name = 'euc'
        
    if embedding_category == 'laplacian':
        em_name = 'lap'
    elif embedding_category == 'Node2Vec':
        em_name = 'n2v'
        
    cur_dir = Path.cwd()
    model_name = training_partition + '_' + testing_partition + '_' + classifier_name + '_model_' + graph_name + em_name + '.pkl'
    model_path = cur_dir.parent / 'MODELS' / model_name
    
    with open(model_path, 'wb') as model_file:
        pickle.dump(classifier, model_file)

    print("Saved ", classifier_name," to", model_path)
    # Print the results
    print(classifier_name, " Model Evaluation Metrics:")
    print("Accuracy:", accuracy)
    print("True Skill Statistics (TSS):", tss)
    print("Heidke Skill Score 1 (HSS1):", hss1)
    print("Heidke Skill Score 2 (HSS2):", hss2)
    print("F1 Score:", f1)
    print("Gilbert Skill Score:", gilbert)
    print("ROC AUC Score:", roc_auc)
    print("********************************************")

In [ ]:
# Train all classifiers (COR-LAP)
for classifier_name in classifiers:
    train_test_classifier_method12('pearson_correlation', 'laplacian', 
                                   X_train_cor_lap, label_vector_cor_train, X_test_cor_lap, 
                                   label_vector_cor_test, classifier_name)

In [ ]:
# Train all classifiers (EUC-LAP)
for classifier_name in classifiers:
    train_test_classifier_method12('euclidian_distance', 'laplacian', 
                                   X_train_euc_lap, label_vector_euc_train, X_test_euc_lap, 
                                   label_vector_euc_test, classifier_name)

In [ ]:
# Train all classifiers (COR-N2V)
for classifier_name in classifiers:
    train_test_classifier_method12('pearson_correlation', 'Node2Vec', 
                                   X_train_cor_N2V, label_vector_cor_train, X_test_cor_N2V, 
                                   label_vector_cor_test, classifier_name)

In [ ]:
# Train all classifiers (EUC-N2V)
for classifier_name in classifiers:
    train_test_classifier_method12('euclidian_distance', 'Node2Vec', 
                                   X_train_euc_N2V, label_vector_euc_train, X_test_euc_N2V, 
                                   label_vector_euc_test, classifier_name)

# Method 1.3: GCN

- You can run GCN experiment for correlation graph data and euclidian graph data from gcn_cor_v1 & gcn_euc_v1 respectively

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Step 1: Get TS features

In [ ]:
def process_data_g(folder_path, graph_category, is_nonflare_train=False):
    data_list = []
    
    if is_nonflare_train:
        csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file.startswith('FQ')]
    else:
        csv_files = natsorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])
        
    # determine number of extracted features 
    if graph_category == 'pearson_correlation':
        n = 23
    elif graph_category == 'euclidian_distance':
        n = 24
    # Loop through the sorted CSV files and load them into dataframes
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        # Use the file name (without extension) as the key in the dictionary
        df = pd.read_csv(file_path, sep='\t')
        # Extract magnetic field parameter values
        first_n_features = df.iloc[:, 1:n+1]
        data_list.append(first_n_features)
    
    # Detect nan entries
    nan_count = 0
    indexes_with_nan = []
    for index, df in enumerate(data_list):
        if df.isna().any().any():
            if index not in indexes_with_nan:
                nan_count += 1
                indexes_with_nan.append(index)
    percentageNaN = (nan_count / len(data_list)) * 100
    
    # Discard nan entries
    data_list = [df for index, df in enumerate(data_list) if index not in indexes_with_nan]
    
    return data_list, percentageNaN

In [ ]:
# For cor, extract TS data
flare_data_list_train_cor, percentageNaN = process_data_g(folder_path_flare_train, 
                                                          'pearson_correlation')
print("Number of DataFrames with NaN values among train flare examples: ", percentageNaN)
nonflare_data_list_train_cor, percentageNaN  = process_data_g(folder_path_nonflare_train, 
                                                              'pearson_correlation',True)
print("Number of DataFrames with NaN values among train nonflare examples: ", percentageNaN)
flare_data_list_test_cor, percentageNaN  = process_data_g(folder_path_flare_test, 'pearson_correlation')
print("Number of DataFrames with NaN values among test flare examples: ", percentageNaN)
nonflare_data_list_test_cor, percentageNaN  = process_data_g(folder_path_nonflare_test, 'pearson_correlation')
print("Number of DataFrames with NaN values among test nonflare examples: ", percentageNaN)


# Combine TS flare and nonflare cor
combined_ts_list_train_cor = np.concatenate([flare_data_list_train_cor, nonflare_data_list_train_cor], axis=0)
combined_ts_list_test_cor = np.concatenate([flare_data_list_test_cor, nonflare_data_list_test_cor], axis=0)

In [ ]:
# For euc, extract TS data
flare_data_list_train_euc, percentageNaN = process_data_g(folder_path_flare_train, 
                                                          'euclidian_distance')
print("Number of DataFrames with NaN values among train flare examples: ", percentageNaN)
nonflare_data_list_train_euc, percentageNaN  = process_data_g(folder_path_nonflare_train, 
                                                              'euclidian_distance',True)
print("Number of DataFrames with NaN values among train nonflare examples: ", percentageNaN)
flare_data_list_test_euc, percentageNaN  = process_data_g(folder_path_flare_test, 
                                                          'euclidian_distance')
print("Number of DataFrames with NaN values among test flare examples: ", percentageNaN)
nonflare_data_list_test_euc, percentageNaN  = process_data_g(folder_path_nonflare_test, 
                                                             'euclidian_distance')
print("Number of DataFrames with NaN values among test nonflare examples: ", percentageNaN)

# Combine TS flare and nonflare cor
combined_ts_list_train_euc = np.concatenate([flare_data_list_train_euc, nonflare_data_list_train_euc], axis=0)
combined_ts_list_test_euc = np.concatenate([flare_data_list_test_euc, nonflare_data_list_test_euc], axis=0)

# Step 2: Get graphs

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import pickle
from pathlib import Path
# this module will bring only the list of graphs 
def process_graphs_m13(partition_name, graph_category, train_or_test):
    f_name = partition_name + '_' + train_or_test
    cur_dir = Path.cwd()
    graph_folder_input = cur_dir.parent / 'DATA' / 'graphs' # For specified graph path
    graph_folder_input = graph_folder_input / graph_category / f_name
    graph_list = []
    labels = []

    # Iterate through all files in the folder
    for filename in os.listdir(graph_folder_input):
        graph_path = os.path.join(graph_folder_input, filename)
        try:
            with open(graph_path, 'rb') as f:
                G = pickle.load(f)
            
            if graph_category == 'pearson_correlation': 
                # IMPORTANT: DROP R-VALUE NODE(24'TH) TO MATCH FT SIZE
                last_node = max(G.nodes())
                G.remove_node(last_node)    
           
            # Determine label based on filename
            if filename.startswith('flare'):
                label = 1
            elif filename.startswith('nonflare'):
                label = 0
        
            # Append degrees and label to lists
            graph_list.append(G)
            labels.append(label)
    
        except Exception as e:
            print(f"Error processing file {filename}: {str(e)}")

    # Convert lists to NumPy arrays
    label_vector = np.array(labels)
    print("Label Vector Shape:", label_vector.shape)
    return graph_list, label_vector

In [ ]:
graph_list_cor_train, label_vector_cor_train=process_graphs_m13(partition_name=training_partition, 
                                                                  graph_category='pearson_correlation', train_or_test='train')
graph_list_cor_test, label_vector_cor_test=process_graphs_m13(partition_name=testing_partition, 
                                                                graph_category='pearson_correlation', train_or_test='test')

In [ ]:
graph_list_euc_train, label_vector_euc_train = process_graphs_m13(partition_name=training_partition, 
                                                                    graph_category='euclidian_distance', train_or_test='train')
graph_list_euc_test, label_vector_euc_test = process_graphs_m13(partition_name=testing_partition, 
                                                                  graph_category='euclidian_distance', train_or_test='test')

# Step 3: Custom dataset

In [ ]:
from torch_geometric.data import Data
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, graph_list, ts_list, labels):
        self.graph_list = graph_list
        self.ts_list = ts_list
        self.labels = labels

    def __len__(self):
        return len(self.graph_list)

    def __getitem__(self, idx):
        graph = self.graph_list[idx]
        ts = self.ts_list[idx]
        label = self.labels[idx]
        
        # For edges, we need to account for both directions of an edge
        node_mapping = {node: i for i, node in enumerate(graph.nodes())}
        edges = list(graph.edges())
        all_edges = [(node_mapping[edge[0]], node_mapping[edge[1]]) for edge in edges]
        all_edges += [(edge[1], edge[0]) for edge in all_edges]
        edge_index = torch.tensor(all_edges, dtype=torch.long).t().contiguous()
        
        data = Data(x=torch.tensor(ts, dtype=torch.float).t(),
                    edge_index=edge_index,
                    y=torch.tensor(label, dtype=torch.float))
        
        return data

In [ ]:
# Create data loaders (for cor)
train_dataset_cor = CustomDataset(graph_list_cor_train, combined_ts_list_train_cor, label_vector_cor_train)
test_dataset_cor = CustomDataset(graph_list_cor_test, combined_ts_list_test_cor, label_vector_cor_test)

In [ ]:
# Create data loaders (for euc)
train_dataset_euc = CustomDataset(graph_list_euc_train, combined_ts_list_train_euc, label_vector_euc_train)
test_dataset_euc = CustomDataset(graph_list_euc_test, combined_ts_list_test_euc, label_vector_euc_test)

# Step 4: Training and testing of GCN

In [ ]:
# Define 2 layer gcn
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Batch
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
from torch_geometric.nn import global_mean_pool

# Define the GCN model with a fully connected layer
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)  # FC for 0-1 classification

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        #print("x.shape ", x.shape)
        #print("edge_index.shape ", edge_index.shape)
      
        #print("X before conv 1", x.shape)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #print("X after conv 2, before pooling", x.shape)
        
        x = x.view(data.num_graphs, int(x.shape[0]/data.num_graphs), x.shape[1])
        x = torch.mean(x, dim=1)
        #x = global_mean_pool(x, batch)
        
        #print("X after pooling, before fc ", x.shape)
        x = self.fc(x)
        #print("X after fc ", x.shape)
        return torch.sigmoid(x) 

# For correlation graphs

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torch_geometric.data import Batch

def custom_collate(batch):
    # Use the Batch class from torch_geometric to handle variable-size graphs
    return Batch.from_data_list(batch)

# Create DataLoader instances withcustom collate function
train_loader = DataLoader(train_dataset_cor, batch_size=64, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset_cor, batch_size=64, shuffle=False, collate_fn=custom_collate)

# Initialize GCN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(in_channels=60, hidden_channels=64, out_channels=32).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

train_losses = []
train_accuracies = []

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        data = batch.to(device) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

        # Update metrics
        total_loss += loss.item()
        total_samples += data.num_graphs
        predicted_labels = (output >= 0.5).float()  # Threshold of 0.5 for binary classification
        #print(predicted_labels, "----", data.y.view(-1, 1))
        correct_predictions += (predicted_labels == data.y.view(-1, 1)).sum().item()
        #print("CORRECT PRED ",correct_predictions, "OUT OF", total_samples)
        
    average_loss_epoch = total_loss / total_samples
    accuracy_epoch = correct_predictions / total_samples
    
    train_losses.append(average_loss_epoch)
    train_accuracies.append(accuracy_epoch)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss_epoch:.4f}, Accuracy: {accuracy_epoch:.4f}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch vs Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epoch vs Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# Evaluate on the test set
model.eval()
true_labels, predicted_labels = [], []

with torch.no_grad():
    for data in tqdm(test_loader, desc='Testing'):
        data = data.to(device)
        output = (model(data) >= 0.5).float()
        true_labels.extend(data.y.cpu().numpy())
        predicted_labels.extend(output.cpu().numpy())

# Convert to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate True Skill Statistics (TSS)
tss = (tp / (tp + fn)) - (fp / (fp + tn))

# Calculate Heidke Skill Score 1 (HSS1)
hss1 = (tp / (tp + fn)) * (2 - (tp + fp) / tp)

# Calculate Heidke Skill Score 2 (HSS2)
hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

# Calculate F1 Score
f1 = f1_score(true_labels, predicted_labels)

# Calculate Gilbert Skill Score
gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (
        tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)
)

# Calculate ROC AUC score
roc_auc = roc_auc_score(true_labels, predicted_labels)

# Print the results
print("GCN Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("True Skill Statistics (TSS):", tss)
print("Heidke Skill Score 1 (HSS1):", hss1)
print("Heidke Skill Score 2 (HSS2):", hss2)
print("F1 Score:", f1)
print("Gilbert Skill Score:", gilbert)
print("ROC AUC Score:", roc_auc)

# For Euclidian graphs

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torch_geometric.data import Batch

def custom_collate(batch):
    # Use the Batch class from torch_geometric to handle variable-size graphs
    return Batch.from_data_list(batch)

# Create DataLoader instances withcustom collate function
train_loader = DataLoader(train_dataset_euc, batch_size=64, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset_euc, batch_size=64, shuffle=False, collate_fn=custom_collate)

# Initialize GCN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(in_channels=60, hidden_channels=64, out_channels=32).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

train_losses = []
train_accuracies = []

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        data = batch.to(device) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

        # Update metrics
        total_loss += loss.item()
        total_samples += data.num_graphs
        predicted_labels = (output >= 0.5).float()  # Threshold of 0.5 for binary classification
        #print(predicted_labels, "----", data.y.view(-1, 1))
        correct_predictions += (predicted_labels == data.y.view(-1, 1)).sum().item()
        #print("CORRECT PRED ",correct_predictions, "OUT OF", total_samples)
        
    average_loss_epoch = total_loss / total_samples
    accuracy_epoch = correct_predictions / total_samples
    
    train_losses.append(average_loss_epoch)
    train_accuracies.append(accuracy_epoch)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss_epoch:.4f}, Accuracy: {accuracy_epoch:.4f}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch vs Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epoch vs Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# Evaluate on the test set
model.eval()
true_labels, predicted_labels = [], []

with torch.no_grad():
    for data in tqdm(test_loader, desc='Testing'):
        data = data.to(device)
        output = (model(data) >= 0.5).float()
        true_labels.extend(data.y.cpu().numpy())
        predicted_labels.extend(output.cpu().numpy())

# Convert to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate True Skill Statistics (TSS)
tss = (tp / (tp + fn)) - (fp / (fp + tn))

# Calculate Heidke Skill Score 1 (HSS1)
hss1 = (tp / (tp + fn)) * (2 - (tp + fp) / tp)

# Calculate Heidke Skill Score 2 (HSS2)
hss2 = (2 * (tp * tn - fp * fn)) / ((tp + fn) * (tn + fn) + (tp + fp) * (tn + fp))

# Calculate F1 Score
f1 = f1_score(true_labels, predicted_labels)

# Calculate Gilbert Skill Score
gilbert = (tp - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)) / (
        tp + tn - (tp + fn) * (tp + fp) / (tp + tn + fp + fn)
)

# Calculate ROC AUC score
roc_auc = roc_auc_score(true_labels, predicted_labels)

# Print the results
print("GCN Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("True Skill Statistics (TSS):", tss)
print("Heidke Skill Score 1 (HSS1):", hss1)
print("Heidke Skill Score 2 (HSS2):", hss2)
print("F1 Score:", f1)
print("Gilbert Skill Score:", gilbert)
print("ROC AUC Score:", roc_auc)

#--------------------------------------------------------------#
# by Onur Vural
# Version Date: January 15, 2024
#--------------------------------------------------------------#